# Final Report: 06/19/2019

## Brief: What was done previously
Previous work discovered multiple mechanisms to improve the accuracy of prediction. Here we quantify the performance of those methods and test them against a standard baseline. Additionally, we quantify learned model's resistence to noise and missing observations.

## Hypothesis

1. Hypothesis 1: Deep recurrent networks are tolerant of sensor noise below a certain magnitude (fixed gaussians per pixel noise)
2. Hypothesis 2: Deep recurrent networks are tolerant of missing samples (random per pixel dropout)
3. Hypothesis 3: Deep recurrent networks are tollerant of missing observations (random per history dropout)




## Summary of Main Results and Discussions



### Experiment 1 results and discussion
Leaned models handle noise suppression impresivly well, dealing with noise with standard deviation twice as large as the range of the sample data. 

### Experiment 2: results and discussion
Put main result and conclusions here. Discuss importance/impact in terms of the project goals.


## Plan for next effort
What will be tested/extended from this week?
    

    


In [1]:
# import packages 
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import os

# Hypothesis 1

Deep recurrent networks are tolerant to sensor noise below a certain magnitude

## Validation accuracy over increasing fixed gaussian sensor noise
We add decreasing magnitudes of gaussian noise to the input and predict the clean future signal. We expect, for
high levels of noise, the model to over-fit to the noise. However, after some threshold we expect the model to learn to recover from small perturbations by learning the underlying distribution.

In [1]:
# Gaussian noise study (fixed input noise)
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, RANDOM_SEED, LARGE_DATASET

# Use a fixed seed for noise    
np.random.seed(RANDOM_SEED)

for s in [2, 1, 0.75, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.0001, 0]:
    noise_data = np.random.normal(size=(360, 279, 1000), scale=scale)
    
    loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, input_noise=noise_data, debug=False)
    
    train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=100000, net_name='lstm_3_cells_20_static_noise_{}'.format(scale))
    
    tf.reset_default_graph()


C:\Users\brandon\source\orbitalMechanics
Instructions for updating:
Colocations handled automatically by placer.
Input shape: (20, ?, 2500)
Output shape: (20, ?, 50, 50)
Encoder input shape: [20, None, 2500]
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Padded input shape: [20, 64, 2500]
[20, 64, 250]
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Summary name Loss Histogram is illegal; using Loss_Histogram instead.
INFO:tensorflow:Summary name Mean Abs Error is illegal; using Mean_Abs_Error instead.
( 0.283889 0 )
( 0.008285959 500 )
0.0033416222 1000
0.

## Explore results

In this test given a 3 layer encoder/decoder with 250 units per layer, we see that the performance of the model is 
resistant to up to 5% noise without any degradation in predictive power. Even with 100% noise the model learned to 
reject some amount of noise and 

In [46]:
# Compare accuracy of model with increasing fixed noise
import os
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.offline.init_notebook_mode(connected=True)
    
# Compare MSE vs magnitude of noise
noise = [2, 1, 0.75, 0.5, 0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.0001, 0]
noise = [noise[-(i+1)] for i in range(len(noise))]
# validation_accuracy = [2.0371e-3, 8.1668e-4, 5.475e-4, 2.8722e-4, 1.0855e-4, 4.5386e-5, 3.1529e-5, 2.7214e-5, 2.0735e-5]
train_accuracy = [1.836e-5, 1.8441e-5, 2.0871e-5, 2.0525e-5, 2.0918e-5, 2.2431e-5, 2.2850e-5, 3.6688e-5, 6.5500e-5, 7.1376e-5, 7.3004e-5, 7.1560e-5, 1.1372e-4]
validation_accuracy = [1.932e-5, 1.9536e-5, 2.1711e-5, 2.1794e-5, 2.1213e-5, 2.3923e-5, 2.4532e-5, 4.2984e-5, 1.0194e-4, 2.9481e-4, 5.4323e-4, 7.5883e-4, 2.0583e-3]

# Create a trace
trace1 = go.Scatter(
    x = noise,
    y = validation_accuracy,
    name="validation"
)
trace2 = go.Scatter(
    x = noise,
    y = train_accuracy,
    name="train"
)

data = [trace1]
layout = go.Layout(
    title="Magnitude of Sensor Noise vs Prediction Accuracy",
    xaxis=dict(
        type='log',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard Deviation of Added Gaussian Noise',
    ),
    yaxis=dict(
        type='log',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean Squared Validation Error over 20 Steps',
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='static_noise_model')



In [47]:
data = [trace1, trace2]
layout = go.Layout(
    title="Magnitude of Sensor Noise vs Prediction Accuracy",
    xaxis=dict(
        type='log',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Standard Deviation of Added Gaussian Noise',
    ),
    yaxis=dict(
        type='log',
        tickmode = 'array',
        autorange=True,
        showexponent = 'all',
        exponentformat = 'e',
        title='Mean Squared Validation Error over 20 Steps',
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='static_noise_model')

In [66]:
# Visualize data
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Make sub-plots for input sequence
i = 0
plt.rc('text', usetex=True)
fig, plots = plt.subplots(1, 8, figsize=(20, 4))

exp_root = './experiments/turbulence/recurrent_scaled_mse'
exp_folders = ['lstm_3_cells_20_static_noise_{}_velocity_and_vorticity_field_1200s.mat_lr0.0005'.format(n) for n in noise if n != 0.0001]
titles = ['Sequence + N $\sig = {} \mu = 0$'.format(n) for n in noise if n != 0.0001]
for directory in exp_folders:

    for file in os.listdir(os.path.join(exp_root, directory)):
        if i < 8:
            if file.endswith('.npz') and file.startswith('inputs'):
                file_path = os.path.join(exp_dir, directory, file)
                with np.load(file_path) as foo:
                    plots[i].imshow(np.reshape(foo['100000'][0,0,:], (50,50)))
                    plots[i].set_title(titles[i])
                    i += 1
                    print(directory)

for ax in fig.get_axes():
    ax.label_outer()
    
plt.show()
                
                



lstm_3_cells_20_static_noise_0_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.0025_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.005_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.01_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.025_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.05_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.1_velocity_and_vorticity_field_1200s.mat_lr0.0005
lstm_3_cells_20_static_noise_0.25_velocity_and_vorticity_field_1200s.mat_lr0.0005


RuntimeError: latex was not able to process the following string:
b'Sequence + N \\\\sig = 0 \\\\mu = 0'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.20 (MiKTeX 2.9.7000 64-bit)
entering extended mode
(C:/Users/brandon/.matplotlib/tex.cache/d8c13150399e14d64f1f9da6540ceca6.tex
LaTeX2e <2018-12-01>
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\article.cls
"
Document Class: article 2018/09/03 v1.4i Standard LaTeX document class
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\size10.clo"
))
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/type1cm\type1cm.
sty")
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\textcomp.st
y"
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.def"
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\ts1enc.dfu"
)))
("C:\Users\brandon\AppData\Local\Programs\MiKTeX 2.9\tex/latex/base\inpu

<Figure size 1440x288 with 8 Axes>

In [48]:
#     try:
#         with np.load(os.path.join(exp_dir, directory, 'label_100000_0.0008166771149262786.npy')) as labels:
#             with np.load(os.path.join(exp_dir, directory, 'pred_100000_0.0008166771149262786.npy')) as predictions:
#                 foo = {key:labels[key].item() for key in labels}
#                 for arr in predictions.keys():
#                     print(predictions[arr].item())
#                     for key, value in dict(predictions[arr].tolist()).items():
#                         print (key,':', value.shape)
#                     # print(dict(labels))
#                     # print(dict(labels[key].tolist())['2000'])
#                     # print(predictions[key])
#     except FileNotFoundError:
#         continue
#     
# 
# trace = go.Heatmap(
#     z=frame,
#     colorscale='Viridis')
# data=[trace]
# py.iplot(data, filename='basic-heatmap')

# Hypothesis 2

Sample level dropout

# Test 1

Description of test 1 of hypothesis 2.


In [70]:
# Missing samples study (random missing pixels)
import numpy as np
import tensorflow as tf

from src.predict_turbulence_recurrent import train
from src.dataLoader.turbulence import Turbulence, RANDOM_SEED, LARGE_DATASET

# Use a fixed seed  
np.random.seed(RANDOM_SEED)

for s in [0.25, 0.1, 0.05, 0.025, 0.01, 0.005, 0.0025, 0.001]:
        
    loader = Turbulence(pred_length=20, dataset_idx=LARGE_DATASET, debug=False)
    
    train(loader=loader, dataset_idx=LARGE_DATASET, num_batches=100000, pixel_dropout=s,
          net_name='lstm_3_cells_20_pixel_dropout_{}'.format(s))
    
    tf.reset_default_graph()

0.25
0.1
0.05
0.025
0.01
0.005
0.0025
0.001
